In [1]:
import os
import dask.dataframe as dd
import datetime as dt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Ouverture des bases de données
df_cct = dd.read_csv("D:\Statapp\data\credit_card_transactions-ibm_v2.csv", dtype={"Errors?" : "object"} )
df_cards = pd.read_csv("D:\Statapp\data\sd254_cards.csv")
df_users = pd.read_csv("D:\Statapp\data\sd254_users.csv")
df_cctu0 = pd.read_csv(r"D:\Statapp\data\User0_credit_card_transactions.csv")

In [3]:
#Un peu de nettoyage

df_cct["Amount"] = df_cct["Amount"].str.replace("$","")
df_cct["Amount"] = df_cct["Amount"].astype(float)


In [4]:
df_cct[['Hours', 'Minutes']] = df_cct.Time.str.split(":", n=1, expand=True) #scinder la colonne Time en 2
df_cct["Hours"] = df_cct["Hours"].astype(int)
df_cct["Minutes"] = df_cct["Minutes"].astype(int)
#Créer une colonne 'Timestamp' : format année, mois, jour, heure, minute:
timestamp = dd.to_datetime(df_cct[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
df_cct["timestamp"] = timestamp.values
df_cct["delta_t_card"] = (timestamp-timestamp.shift()) #calculer l'intervalle de temps entre
#deux transactions de la même carte et du même user

#Convertir delta_t_card en secondes et mettre à 0 les temps négatifs 
#(ici, dataframe ordonné en fonction de l'user et de la carte, et non de la chronologie):
df_cct["delta_t_s_card"] = df_cct["delta_t_card"].dt.total_seconds().fillna(0).astype(int)
df_cct["delta_t_s_card"] = df_cct["delta_t_s_card"].mask(df_cct["delta_t_s_card"] < 0, 0)



In [9]:
import numpy as np
import struct

In [10]:
df_cct['timestamp_int'] = ((df_cct['timestamp'] -
                  df_cct['timestamp'].min())/np.timedelta64(1,'s')
                ).astype(int)

df_cct['new_column'] = df_cct.apply(lambda r: struct.pack("ll",r.timestamp_int,r.User),axis=1)
df_cct = df_cct.set_index('new_column')
df_cct = df_cct.map_partitions(lambda x: x.sort_index())

In [ ]:
#Réordonner les transactions en fonction du User et de la chronoogie de ses transactions : 
df_cct = df_cct.sort_values(["User", "Year","Month","Day","Hours","Minutes"], ascending=[True,True,True,True,True,True]).reset_index(drop=True)
timestamp2 = dd.to_datetime(df_cct[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
df_cct['delta_t'] = (timestamp2-timestamp2.shift()) #différence de temps entre deux transactions du même user
df_cct["delta_t_s"] = df_cct["delta_t"].dt.total_seconds().fillna(0).astype(int) #pour la première
#transaction, fixer le delta_t_s à 0
df_cct = df_cct[['User', 'Card', 'timestamp', 'Amount', 'Use Chip',
    'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
    'Errors?', 'Is Fraud?', 'delta_t_s','delta_t_s_card']] #ordonner et sélectionner les colonnes

In [4]:
#Création des variables 'timestamp', 'delta_t_s' et 'delta_t_s_card'
def time(dataframe):
   dataframe[['Hours', 'Minutes']] = dataframe.Time.str.split(":", n=1, expand=True) #scinder la colonne Time en 2
   dataframe["Hours"] = dataframe["Hours"].astype(int)
   dataframe["Minutes"] = dataframe["Minutes"].astype(int)
   #Créer une colonne 'Timestamp' : format année, mois, jour, heure, minute:
   timestamp = dd.to_datetime(dataframe[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
   dataframe["timestamp"] = timestamp.values
   dataframe["delta_t_card"] = (timestamp-timestamp.shift()) #calculer l'intervalle de temps entre
   #deux transactions de la même carte et du même user

   #Convertir delta_t_card en secondes et mettre à 0 les temps négatifs 
   #(ici, dataframe ordonné en fonction de l'user et de la carte, et non de la chronologie):
   dataframe["delta_t_s_card"] = dataframe["delta_t_card"].dt.total_seconds().fillna(0).astype(int)
   dataframe["delta_t_s_card"] = dataframe["delta_t_s_card"].mask(dataframe["delta_t_s_card"] < 0, 0)

   #Réordonner les transactions en fonction du User et de la chronoogie de ses transactions : 
   dataframe = dataframe.sort_values(["User", "Year","Month","Day","Hours","Minutes"], ascending=[True,True,True,True,True,True]).reset_index(drop=True)
   timestamp2 = dd.to_datetime(dataframe[["Year",'Month','Day','Hours','Minutes']], format = '%Y:%M:%D%:%H:%M')
   dataframe['delta_t'] = (timestamp2-timestamp2.shift()) #différence de temps entre deux transactions du même user
   dataframe["delta_t_s"] = dataframe["delta_t"].dt.total_seconds().fillna(0).astype(int) #pour la première
   #transaction, fixer le delta_t_s à 0
   dataframe = dataframe[['User', 'Card', 'timestamp', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?', 'delta_t_s','delta_t_s_card']] #ordonner et sélectionner les colonnes
   
   return dataframe


